In [3]:
from glob import glob
import os

# Text clustering and classification #

- Tokenization
- Bag-of-words vectors
- Cosine similarity
- Stopwords and TF-IDF
- K-nearest neighbors

Let's just pick a single story to start

In [4]:
basedir = 'aa'
storyname = os.path.join(basedir,'TheLiberator/Issue of October 21, 1859/story028.txt')

In [5]:
with open(storyname,'r') as fin:
    print fin.readlines()

[' MARIUS R. ROBINSON, of Ohio, is an Agent of theAmerican Anti-Slavery Society, and as such is commendedto all friends of the Society, and of uncompromisinganti-slavery. As editor of the (Ohio) Anti-SlaveryBugle, and as a clear, earnest, and impressivespeaker, his services have been of the greatest valueto the cause, and have entitled him to the fullest confidenceand respect of its friends. In full apprehensionof the principles of Anti-Slavery, in faithful applicationof them, and in a fair and courteous spirit toopponents, he is surpassed by no one. ARIUS OBINSON American Anti-Slavery Society Anti-SlaveryBugle Mr. Robinson is at present laboring in WesternNew York. He will receive subscriptions to the NationalAnti-Slavery Standard, and other Anti-Slaverypapers, and donations to the American Anti-slaverySociety. NationalAnti-Slavery Standard For the Executive Committee. \n']


## Tokenization ##

First, we need to divide the text into **tokens**. We can do that by splitting on whitespace.

In [6]:
def getTokens(storyname):
    output = []
    with open(storyname,'r') as fin:
        for line in fin:
            output.extend(line.split())
    return output

In [7]:
print getTokens(storyname)[:20]

['MARIUS', 'R.', 'ROBINSON,', 'of', 'Ohio,', 'is', 'an', 'Agent', 'of', 'theAmerican', 'Anti-Slavery', 'Society,', 'and', 'as', 'such', 'is', 'commendedto', 'all', 'friends', 'of']


Notice how the token ```Society,``` has a comma in it. We'd like to do better than split on whitespace, so we can remove these pieces. NLTK, *the natural language toolkit*, has better tokenizers. They're not complicated (for English!), they're just finely-tuned regular expressions.

In [8]:
from nltk.tokenize import WordPunctTokenizer

In [9]:
tokenizer = WordPunctTokenizer()

In [10]:
def getTokensBetter(storyname):
    output = []
    with open(storyname,'r') as fin:
        for line in fin:
            output.extend(tokenizer.tokenize(line.rstrip()))
    return output

In [11]:
print getTokensBetter(storyname)[:20]

['MARIUS', 'R', '.', 'ROBINSON', ',', 'of', 'Ohio', ',', 'is', 'an', 'Agent', 'of', 'theAmerican', 'Anti', '-', 'Slavery', 'Society', ',', 'and', 'as']


## Bag-of-words vectors ##

Next, we're going to build *bag of words* vectors. Each vector stores the counts of all words in a given document.

- To build such a vector, we need to know what the words are.  
- So we will make two passes, first to build the vocabulary and then to build the vectors.
- Python's Counter class makes this easier.

In [12]:
from collections import Counter # for the dictionary
import numpy as np # for vectors

In [13]:
file_glob = glob(os.path.join(basedir,'*','*','story*'))

In [14]:
all_counts = Counter()
for filename in file_glob: #iterate through all the files
    for tok in getTokensBetter(filename): #for each token
        all_counts[tok] += 1

In [15]:
print 'Length:',len(all_counts)
print all_counts.most_common(10)

Length: 46111
[(',', 48149), ('the', 34964), ('.', 30711), ('of', 22025), ('and', 17525), ('to', 14544), ('a', 10302), ('in', 9764), ('that', 5661), ('is', 5291)]


To map from words to their positions in the vector, we will use an *inverted index*.

This is just a dictionary from words to unique numbers. We'll focus on the 1000 most common words.

In [16]:
inv_index = {word[0]:loc for loc,word in enumerate(all_counts.most_common(1000))}

In [17]:
for word in ['it','was','the','best','of','times']:
    print word,inv_index[word]

it 14
was 13
the 1
best 202
of 3
times 501


In [18]:
print inv_index['laptop']

KeyError: 'laptop'

Now let's build the bags of words.

We'll store it as a numpy array. Note that we are storing 700K integers. If the data were a few orders of magnitude bigger, it would be hard to store it this way. Instead, we would need to account for the **sparsity structure** of linguistic data, which is that 

In [19]:
N = len(file_glob)
V = len(inv_index)
print N,V
X = np.zeros((N,V)) # NOTE! we couldn't do it this way if N or V were much bigger. We'd need a sparse matrix.

705 1000


In [20]:
for i,filename in enumerate(file_glob):
    for tok in getTokensBetter(filename):
        if tok in inv_index:
            X[i,inv_index[tok]] += 1

In [21]:
print 'num nonzero:', len(X.nonzero()[0])
print 'size of X:',N*V
print 'density:', len(X.nonzero()[0])/float(N*V)

num nonzero: 109392
size of X: 705000
density: 0.155165957447


Here is a bag of words for one document:

In [22]:
X[10,:]

array([ 93.,  40.,  59.,  25.,  27.,  31.,  35.,  16.,  11.,  15.,  29.,
        13.,  94.,  10.,  18.,   4.,   7.,   5.,  12.,   4.,  12.,   7.,
         2.,   0.,  17.,   6.,   5.,  10.,   4.,   5.,   2.,   8.,   4.,
         4.,  13.,  30.,   7.,   1.,   2.,   2.,   5.,   6.,   0.,   2.,
         4.,  40.,   4.,  41.,   4.,   2.,  10.,   2.,   4.,  38.,   0.,
         0.,   0.,   7.,   7.,   1.,  14.,   4.,  18.,   2.,   2.,   4.,
         9.,   0.,   1.,   2.,   0.,   0.,   2.,   2.,   3.,   4.,   0.,
         2.,   3.,   3.,   1.,   2.,   0.,   2.,   2.,   6.,   5.,   0.,
         4.,   2.,   2.,   3.,   0.,   1.,   2.,   8.,   2.,   0.,   3.,
         1.,   2.,   0.,   1.,   1.,   4.,   2.,   6.,   3.,   1.,   1.,
         2.,   0.,   3.,   0.,   2.,   5.,   1.,   4.,   1.,   0.,   4.,
         5.,   0.,   0.,   3.,   2.,   0.,   4.,   0.,   2.,   2.,   1.,
        13.,   0.,   0.,   2.,   0.,   0.,   1.,   0.,   3.,   0.,   0.,
         0.,   4.,   4.,   0.,   0.,   5.,   0.,   

## Scikit-learn ##

Now that you know how to do it by hand, we can talk about libraries that do it for you.

```Scikit-learn``` is a library of machine learning functions for python. 
Its ```vectorizer``` classes handle nearly the entire process of converting text into bag-of-word vectors.

See [here](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) for more documentation.

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

In [117]:
all_files = sorted(file_glob)

In [118]:
data = {filename:' '.join(getTokensBetter(filename)) for filename in all_files}

In [121]:
print data.keys()[:3]

['aa/TheLiberator/Issue of October 14, 1859/story073.txt', 'aa/TheNationalEra/Issues of October 06, 1859/story006.txt', 'aa/TheLiberator/Issue of October 14, 1859/story049.txt']


In [122]:
vect = CountVectorizer(max_features=1000,decode_error='replace',lowercase=False,tokenizer=tokenizer.tokenize)

In [123]:
x2 = vect.fit_transform([data[storyname] for storyname in all_files])

In [124]:
X2

<705x1000 sparse matrix of type '<type 'numpy.int64'>'
	with 109258 stored elements in Compressed Sparse Row format>

Almost the same, but not quite -- the homebrew version has 109392 non-zeros. 

I've tried to make the preprocessing identical, but it's possible there are some subtle differences.

Note also that we are getting a sparse matrix here, instead of an array. This is the more scalable data type that I was talking about.

# Cosine similarity #

From wikipedia:

\begin{align}
\mathbf{a} \cdot \mathbf{b} = 
& || \mathbf{a} || \times || \mathbf{b} || \times \cos \theta_{\mathbf{a},\mathbf{b}}\\
\cos \theta_{\mathbf{a},\mathbf{b}} = & \frac{\mathbf{a} \cdot \mathbf{b}}{||\mathbf{a}||\times ||\mathbf{b}||}
\end{align}

In [143]:
def cossim(a,b):
    return np.dot(a,b) / (1e-6 + np.linalg.norm(a) * np.linalg.norm(b))

In [127]:
x3 = np.array(x2.todense())

In [128]:
print np.dot(x3[0,:],x3[5,:])
print np.linalg.norm(x3[0,:]), np.linalg.norm(x3[5,:])
print cossim(x3[0,:],x3[5,:])

575
30.7896086367 36.2904946232
0.51460119281


# K-nearest neighbors clustering #

In K-nearest neighbors (KNN), each point is placed in the same cluster as the majority of its $K$-nearest neighbors. Nearness is defined in terms of cosine similarity.

In [144]:
sims = np.zeros((N,N))
for i,x_i in enumerate(x3):
    for j,x_j in enumerate(x3[i:,:]):
        sim_ij = cossim(x_i,x_j)
        sims[i,j+i] = sim_ij
        sims[j+i,i] = sim_ij

Let's see how good this similarity function is.

In [165]:
print 'Query:',data[all_files[10]][:100]
print ''
for i,filenum in enumerate(sims[10,].argsort()[::-1][1:4]):
    print 'sim',i,data[all_files[filenum]][:100]
print ''
for i,filenum in enumerate(sims[10,].argsort()[:10]):
    if len(data[all_files[filenum]]) > 20:
        print 'dif',i,data[all_files[filenum]][:100]    

Query: � �� About a month ago , a cargo of slaveswas landed at Trinidad de Cuba . One of theinspectors , a 

sim 0 PARIS PARIS [ FROM OUR OWN CORRESPONDENT .] Count de Morny ' s address and what is thought of it on 
sim 1 MISS MULOCH . MISS MULOCH . A RECENT tourist thus gives her impressions of a visit to Miss Muloch : 
sim 2 THE EMPRESS OF FRANCE . THE EMPRESS OF FRANCE . IT is so seldom that either birth or charm puts a pr

dif 7 Collection by C . L . Remond . Collection by C . L . Remond . Table Table Collections by Mrs . F . H
dif 8 CHESS . CHESS . PROBLEM No . 213 . � �� By . A . J . H ., Kewanee , Ill . White to play and checkmat
dif 9 No order attended to unless the cash accompanies it . All persons requiring answers by mail must sen


Looks sort of reasonable? So:

1-nearest-neighbor clustering algorithm:

- Set $\text{cluster}(i) = rand(K)$ for all $i$
- While not tired
    - For instance i in $1\ldots N$
        - Set $\text{cluster}(i) = \text{cluster}(j)$ for $j = \text{argmax}_j(sim(i,j))$

In [173]:
from scipy.stats import mode

In [224]:
def doKNN(sims,C,K=3,n_its = 5):
    K = 3 #3-nearest-neighrbods
    y = np.random.randint(0,C,N) #initialization
    for _ in range(n_its):
        for i in range(N): #would be better to randomize the order
            neighbor_clusters = np.array([y[j] for j in sims[i,:].argsort()[::-1][1:1+K]]) #get clusters of most sim nodes
            most_sim = mode(neighbor_clusters)
            y[i] = most_sim.mode[0]
    return y

In [198]:
y = doKNN(sims,30)

Let's look at the clusters!

In [202]:
def showCluster(cluster,y):
    clust_idxs = [i for i,y_i in enumerate(y) if y_i==cluster]
    for c_idx in clust_idxs:
        if len(data[all_files[c_idx]]) > 20:
            print data[all_files[c_idx]][:100]    

In [203]:
showCluster(10,y)

� �� Forty - five slaves were baptised at Richmond , Va ., on a recent Sunday .
PARLOR GOSSIP FOR THE LADIES . PARLOR GOSSIP FOR THE LADIES . Who to Choose for a Wife . � �� The Ge
FLORENCE DE LACY ; FLORENCE DE LACY ; OR , QUICKSANDS AND WHIRLPOOLS . QUICKSANDS AND WHIRLPOOLS . A
FACT , FUN AND FANCY . FACT , FUN AND FANCY . TIPSY WIT . � �� Sheridan was staggering home one nigh
FLORENCE DE LACY ; FLORENCE DE LACY ; OR , QUICKSANDS AND WHIRLPOOLS . QUICKSANDS AND WHIRLPOOLS . A
FLORENCE DE LACY ; FLORENCE DE LACY ; OR , QUICKSANDS AND WHIRLPOOLS . QUICKSANDS AND WHIRLPOOLS . A
FACT , FUN AND FANCY . FACT , FUN AND FANCY . AWFUL ! � �� Why is the east wind like a famous Americ
COLONEL EAU DE VIE . COLONEL EAU DE VIE . AT one of our principal hotels the other evening , a party
THE SONG OF TO - MORROW . THE SONG OF TO - MORROW . THERE ' S a To - morrow for all of us� �� � �� T
FLORENCE DE LACY ; FLORENCE DE LACY ; OR , QUICKSANDS AND WHIRLPOOLS . QUICKSANDS AND WHIRLPOOLS . A
Tiour God a

In [206]:
showCluster(1,y)

According to promise , we publish in ourpresent number Mr . SMITH ' S letter to Mr . THOMAS in favor
REV . GEO . B . CHEEVER BEFORE THE AMERICANBOARD OF COMMISSIONERS FOR FOREIGNMISSIONS . � �� This ve
The United States are just now being honoredby the presence of three distinguishedclergymen from the
WASHINGTON , July 14 , 1859 . ASHINGTON Mrs . Lucy Stone� �� Dear Madam : Your kindletter of the 8th
� �� Henry Mitchell , a colored man , is in troubledin Boston for forging a note of $ 1 , 062 on Ger
� �� The Charleston Mercury says that theobject of John Mitchel in going to France is toget Napoleon
When Mr . SEWARD said in his Rochesterspeech , a year ago , that there is an irrepressibleconflict b
We had hoped to have been able to havelaid before our readers , in our present number , the resoluti
' He has preached both the ballot and the bulletas the means by which slavery is to be destroyedand 
Resolved , That we approve and reiterate theprinciples laid down in the Cincinnati platform

# Making it better: TF-IDF #

In [207]:
from sklearn.feature_extraction.text import TfidfTransformer

In [209]:
transformer = TfidfTransformer()

In [220]:
x3_tfidf = np.array(transformer.fit_transform(x3).todense())

In [221]:
x3_tfidf[0,:50]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.26164387,
        0.        ,  0.        ,  0.06638311,  0.6559655 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.09130047,  0.        ,
        0.        ,  0.        ,  0.10496208,  0.04579454,  0.        ,
        0.        ,  0.        ,  0.08646348,  0.09999254,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [235]:
sims_tfidf = np.zeros((N,N))
for i,x_i in enumerate(x3_tfidf):
    for j,x_j in enumerate(x3_tfidf[i:,:]):
        sim_ij = cossim(x_i,x_j)
        sims_tfidf[i,j+i] = sim_ij
        sims_tfidf[j+i,i] = sim_ij

In [236]:
y_tfidf = doKNN(sims_tfidf,30,n_its=10)

In [241]:
showCluster(0,y_tfidf)

THE GREAT EASTERN STEAMSHIP PICTORIAL , THE GREAT EASTERN STEAMSHIP PICTORIAL , BY FRANK LESLIE . TH
THE PROPELLER ENGINE - ROOM OF THE GREAT EASTERN . THE PROPELLER ENGINE - ROOM OF THE GREAT EASTERN 
THE ACCIDENT ON BOARD THE GREAT EASTERN . THE ACCIDENT ON BOARD THE GREAT EASTERN . Table Table PLAN
HYDRAULIC RAMS USED AT THE LAUNCHING OF THE GREAT EASTERN . HYDRAULIC RAMS USED AT THE LAUNCHING OF 
THE BATTLE OF THE PEIHO . THE BATTLE OF THE PEIHO . THE engraving of the attack by the Allied forces
THE SONS OF MALTA . THE SONS OF MALTA . Table Table TOE DIAMOND WEDDING� �� SCENE IN THE CATHEDRAL� 
All letters for the National Era must be addressed to Mrs . M . L . BAILEY , National Era , Washingt
TO ADVERTISERS . Business men will find it greatly to their advantage to advertise in the Era . Mess
All letters for the National Era must be addressed to Mrs . M . L . BAILEY , National Era , Washingt


In [246]:
showCluster(5,y_tfidf)

PETERBORO , Sept 19 , 1859 . ETERBORO Mr . FREDERICK DOUGLASS : MY DEARFRIEND : � �� I have just rea
� �� There are now ten Anti - Slavery papersprinted in the Slave States ( in English ,) besideeight 
OCTOBER NUMBER OCTOBER NUMBER FRANK LESLIE ' S FRANK LESLIE ' S NEW FAMILY MAGAZINE . THE October nu
WILLIAM HOLMES , THE MISSING MAN . WILLIAM HOLMES , THE MISSING MAN . AT the earnest request of the 
SIX HOURS AT COURT . SIX HOURS AT COURT . WITH a negative capital of eight hundred dollars , a borro
FRANK LESLIE ' S FRANK LESLIE ' S Illustrirte Zeitung , Illustrirte Zeitung , Illustrirte Zeitung ,
� � I DON ' T think you seem very glad to gel home , Anne , I must say . � � Mrs . Harrison ' E face
� � A LETTER from father ! � � shouted Robert Murray , as he came running up the short gravel - walk
� � With wild surprise , As It to marble struck , devoid of sense , A stupid moment motionless she s
PROVIDENCE , June 7 , 1859 . June 7 , L . A . GODEY , ESQ . DEAR SIR : I was so much pleased